In [ ]:
import pandas as pd
import math
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone
import pyodbc
from sqlalchemy import Column, Date, Integer, String, Numeric, create_engine, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, LargeBinary
from sqlalchemy.orm import mapper, registry, Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func, MetaData, Table
from sqlalchemy import create_engine, func, MetaData, Table, select, Column, Integer, String
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, balanced_accuracy_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sqlalchemy import inspect
import scipy
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sqlalchemy.orm import relationship
import pandas as pd
import math
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone
import pyodbc
from sqlalchemy import Column, Date, Integer, String, Numeric, create_engine, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, func
from sqlalchemy.orm import mapper, registry, Session
import pandas as pd
import math
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone
import pyodbc
from sqlalchemy import Column, Date, Integer, String, Numeric, create_engine, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, LargeBinary
from sqlalchemy.orm import mapper, registry, Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func, MetaData, Table
from sqlalchemy import create_engine, func, MetaData, Table, select, Column, Integer, String
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, balanced_accuracy_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sqlalchemy import inspect
import scipy
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
#SQLAlchemy Setup
def get_engine():
    SERVER = 'localhost'
    DATABASE = 'metmast'
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect=DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes;')
    return engine

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)
mapper_registry = registry()

In [ ]:
#Load Trainingsdata
def loadData(tableName):
    # Get the train_data table
    table = Table(tableName, metadata)
    session = Session(bind=engine)    
    batch_size = 500
    query = session.query(table)
    #Load Query into Dataframe
    df = pd.DataFrame()
    for batch in pd.read_sql_query(query.statement, engine, chunksize=batch_size):
        df = pd.concat([df, batch], ignore_index=True)
        excluded_columns=["id"]
        if excluded_columns:
            df = df.drop(columns=excluded_columns)    
    return df

train_X = loadData("train_data")
train_y = loadData("train_data_label").drop(columns=["train_data_id", "Class"]) 
test_X = loadData("test_data")
test_y = loadData("test_data_label").drop(columns=["test_data_id", "Class"]) 

In [ ]:
#Load Preset
session = Session(bind=engine)  
query = session.query(metadata.tables["hyperparameter"])
paras = pd.read_sql_query(query.statement, engine, chunksize=1)


In [ ]:
paras

In [ ]:
dtr = DecisionTreeClassifier(
        max_depth = args["max_depth"],
        min_samples_leaf = args["min_samples_leaf"],
        random_state = args["random_state"],
        max_features = args["max_features"],
        criterion = args["criterion"],
        #normalize = args["normalize"],
    )

In [ ]:
#Methods
def update_values(obj, ref_class, session, **kwargs):
    #clone = ref_class()
    for column in ref_class.__table__.columns:
        if(column.name in kwargs):
            setattr(obj, column.name, kwargs[column.name])
        #else:
            #setattr(clone, column.name, getattr(obj, column.name))
    #session.delete(obj)
    #session.add(clone)
    session.commit() 
    return obj
def get_table_by_name(name):
    inspector = inspect(engine)
    return inspector.get_table(name)

def get_feature_id_by_name(Base, name):
    query = session.query(metadata.tables["feature"])
    df = pd.DataFrame()
    for batch in pd.read_sql_query(query.statement, engine, chunksize= 5):
        df = pd.concat([df, batch], ignore_index=True)
    for index, row in df.iterrows():
        if(row["name"]==name):
            return row["id"]
        
def getNextIdForTable(Base, name):
    table = Base.classes[name]
    return session.query(table.id).count() + 1 
  
def create_object(Base, class_name, **kwargs):
    mapped_class = Base.classes[class_name]
    # Create an instance of the mapped class
    obj = mapped_class()
    # Set the object's attributes
    for key, value in kwargs.items():
        setattr(obj, key, value)
    return obj